Načtení dat

In [1]:
import pandas as pd
data = pd.read_csv("diabetes_prediction_dataset.csv")
data

,gender,age,hypertension,heart_disease,smoking_history,bmi,HbA1c_level,blood_glucose_level,diabetes
0,Female,80.0,0,1,never,25.19,6.6,140,0
1,Female,54.0,0,0,No Info,27.32,6.6,80,0
2,Male,28.0,0,0,never,27.32,5.7,158,0
3,Female,36.0,0,0,current,23.45,5.0,155,0
4,Male,76.0,1,1,current,20.14,4.8,155,0
...,...,...,...,...,...,...,...,...,...
99995,Female,80.0,0,0,No Info,27.32,6.2,90,0
99996,Female,2.0,0,0,No Info,17.37,6.5,100,0
99997,Male,66.0,0,0,former,27.83,5.7,155,0
99998,Female,24.0,0,0,never,35.42,4.0,100,0


# Rozdělení na training a testing set

In [2]:
from sklearn.model_selection import train_test_split
X = data.drop('diabetes', axis=1)
y = data['diabetes']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print("Training set: ", X_train.shape)
print("Testing set: ", X_test.shape)

KeyboardInterrupt: 

# Doplnění chybějících pozorování pro smoking_history

In [ ]:
print(X_train['smoking_history'].value_counts())
print("celkem: " ,X_train['smoking_history'].shape[0])

smoking_history
No Info        28667
never          28065
former          7510
current         7383
not current     5173
ever            3202
Name: count, dtype: int64
celkem:  80000


In [ ]:
import numpy as np
value_counts = X_train[X_train['smoking_history'] != 'No Info']['smoking_history'].value_counts()
total_count = value_counts.sum()
distribution = (X_train['smoking_history'].value_counts() / total_count).drop('No Info')
random= np.random.choice(distribution.index,X_train[X_train['smoking_history'] == 'No Info'].shape[0] ,p=distribution.values)

X_train['smoking_history'].loc[X_train['smoking_history'] == 'No Info'] = random


/var/folders/z_/k1l7w3xs3ld9scbb1v7zl7sm0000gn/T/ipykernel_23037/920219924.py:7: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  X_train['smoking_history'].loc[X_train['smoking_history'] == 'No Info'] = random
/var/folders/z_/k1l7w3xs3ld9scbb1

In [ ]:
print(X_train['smoking_history'].value_counts())
print("celkem: " ,X_train['smoking_history'].shape[0])

smoking_history
never          43739
former         11686
current        11481
not current     8082
ever            5012
Name: count, dtype: int64
celkem:  80000


In [ ]:
#Jednodušší doplnění módusem
#X_train['smoking_history'] = X_train['smoking_history'].replace('No Info', str(X_train['smoking_history'].mode()[0]))
#X_train['smoking_history']

# Upsampling

In [ ]:
#upsample the testing data to contain more samples with diabetes = 1
from sklearn.utils import resample
X_train['diabetes'] = y_train
#Separate the features and target variable
X,y= X_train.drop('diabetes', axis=1), X_train['diabetes']
#Resampling the minority class (diabetes = 1) by up-sampling
X_up, y_up = resample(X[y==1], y[y==1], replace=True, n_samples=X[y==0].shape[0], random_state=42)
#Combining the upsampled data with the original data of the majority class
X_resampled = pd.concat([X[y==0], X_up])
y_resampled = pd.Series(np.concatenate([y[y==0], y_up]))
print("Počet kde diabetes = 1 po upsamplingu: ", y_resampled.sum()) 

X_train = X_resampled
y_train = y_resampled

Počet kde diabetes = 1 po upsamplingu:  73208


# Odstranění dat

Odstranění záznamů, kde je pohlaví určeno jako Other. Dataset obsahuje těchto instancí jen 16.

In [ ]:
print ("Počet pro pohláví 'Other'",X_train[X_train['gender'] == 'Other'].shape[0])
X_train = X_train.drop(X_train[X_train['gender'] == 'Other'].index)

Počet pro pohláví 'Other' 16


# Převedení kategoriálních hodnot na binární

## Gender na binarní reprezentaci

In [ ]:
X_train = pd.get_dummies(X_train, columns=['gender'])

X_train

,age,hypertension,heart_disease,smoking_history,bmi,HbA1c_level,blood_glucose_level,gender_Female,gender_Male
75220,73.0,0,0,former,24.77,3.5,80,False,True
44966,38.0,0,0,ever,24.33,4.0,158,True,False
13568,26.0,0,0,not current,18.87,5.0,100,True,False
92727,61.0,1,0,current,22.11,4.5,85,True,False
51349,34.0,0,0,never,19.46,5.7,126,True,False
...,...,...,...,...,...,...,...,...,...
92511,28.0,0,0,ever,40.10,7.5,200,True,False
45312,49.0,1,0,never,25.99,5.7,159,False,True
70088,65.0,0,1,former,28.72,6.6,300,False,True
38192,69.0,0,1,not current,31.32,9.0,160,False,True


## Smoking_history na binární reprezentaci

In [ ]:
X_train = pd.get_dummies(X_train, columns=['smoking_history'])

X_train

,age,hypertension,heart_disease,bmi,HbA1c_level,blood_glucose_level,gender_Female,gender_Male,smoking_history_current,smoking_history_ever,smoking_history_former,smoking_history_never,smoking_history_not current
75220,73.0,0,0,24.77,3.5,80,False,True,False,False,True,False,False
44966,38.0,0,0,24.33,4.0,158,True,False,False,True,False,False,False
13568,26.0,0,0,18.87,5.0,100,True,False,False,False,False,False,True
92727,61.0,1,0,22.11,4.5,85,True,False,True,False,False,False,False
51349,34.0,0,0,19.46,5.7,126,True,False,False,False,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
92511,28.0,0,0,40.10,7.5,200,True,False,False,True,False,False,False
45312,49.0,1,0,25.99,5.7,159,False,True,False,False,False,True,False
70088,65.0,0,1,28.72,6.6,300,False,True,False,False,True,False,False
38192,69.0,0,1,31.32,9.0,160,False,True,False,False,False,False,True


##Import pro clustering

In [11]:
from sklearn.cluster import KMeans
from sklearn.datasets import make_blobs
import matplotlib.pyplot as plt

ModuleNotFoundError: No module named 'matplotlib'